# Meta-Llama-3.1-70B-Instruct-AWQ-INT4 (vLLM)

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url="http://195.242.13.129:8001/v1",
    api_key="token-abc123",
)

db_type = "mysql"
user_question = "What is the ticker symbol, month, average closing price, highest price, lowest price, and MoMC for each ticker by month?"

# instructions = "MoMC = month-over-month change in average closing price = (avg_close_given_month - avg_close_previous_month) / avg_close_previous_month for each ticker symbol each month. Recall that we want the symbol, and not just the ticker id."

instructions = """NPM (Net Profit Margin) = (Total Income from Sales - (Tax + Commission Expenses)) / Total Income from Sales * 100, for recent transactions. TAC = Total Active Customers who have recently joined ACP = Average Closing Price of tickers over a recent period MoMC = month-over-month change in average closing price = (avg_close_given_month - avg_close_previous_month) / avg_close_previous_month for each ticker each month"""
table_metadata_string = """-- Table to store daily price data for stocks
CREATE TABLE broker.sbDailyPrice (
    sbDpDate DATETIME,                    -- Date of the daily price
    sbDpTickerId INT,                     -- Foreign key to the ticker
    sbDpClose DECIMAL(10, 2),             -- Closing price of the stock
    sbDpHigh DECIMAL(10, 2),              -- Highest price of the day
    sbDpLow DECIMAL(10, 2),               -- Lowest price of the day
    PRIMARY KEY (sbDpDate, sbDpTickerId), -- Composite key on date and ticker ID
    FOREIGN KEY (sbDpTickerId) REFERENCES broker.sbTicker(sbTickerId)
);

-- Table to store stock ticker symbols
CREATE TABLE broker.sbTicker (
    sbTickerId INT PRIMARY KEY,            -- Unique ID for each ticker
    sbTickerSymbol VARCHAR(10) NOT NULL    -- Stock ticker symbol
);"""
k_shot_prompt = ""

completion = client.chat.completions.create(
    model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    messages=[
        {
            "role": "system",
            "content": f"Your role is to convert a user question to a {db_type} query, given a database schema.",
        },
        {
            "role": "user",
            "content": f"Generate a SQL query that answers the question `{user_question}`.\n{instructions}\nThis query will run on a database whose schema is represented in this string:\n{table_metadata_string}\n{k_shot_prompt}\nReturn only the SQL query, and nothing else.",
        },
    ],
    max_tokens=500,
    temperature=0.1,
)

print(completion.choices[0].message.content)

```sql
WITH MonthlyPriceData AS (
    SELECT 
        T.sbTickerSymbol,
        YEAR(DP.sbDpDate) AS Year,
        MONTH(DP.sbDpDate) AS Month,
        AVG(DP.sbDpClose) AS AverageClose,
        MAX(DP.sbDpHigh) AS HighestPrice,
        MIN(DP.sbDpLow) AS LowestPrice
    FROM 
        broker.sbDailyPrice DP
    JOIN 
        broker.sbTicker T ON DP.sbDpTickerId = T.sbTickerId
    GROUP BY 
        T.sbTickerSymbol,
        YEAR(DP.sbDpDate),
        MONTH(DP.sbDpDate)
),
MonthlyPriceDataWithMoMC AS (
    SELECT 
        sbTickerSymbol,
        Year,
        Month,
        AverageClose,
        HighestPrice,
        LowestPrice,
        (AverageClose - LAG(AverageClose) OVER (PARTITION BY sbTickerSymbol ORDER BY Year, Month)) / LAG(AverageClose) OVER (PARTITION BY sbTickerSymbol ORDER BY Year, Month) AS MoMC
    FROM 
        MonthlyPriceData
)
SELECT 
    sbTickerSymbol,
    Year,
    Month,
    AverageClose,
    HighestPrice,
    LowestPrice,
    MoMC
FROM 
    MonthlyPriceDataWithMoM

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url="http://172.210.166.80:8001/v1",
    api_key="token-abc123",
)

system_prompt = "You are a helpful assistant"

user_prompt = """
Your task is to write sentences (maximum 5 sentences) to describe a table.
Make sure that the sentences you generated are brief, concise, and contain only essential information.
 
Sample 1 of Expected Answer:
A fact table from SAP that is a core component of the Financial Accounting (FI) module, responsible for storing individual line item data related to financial transactions. It holds crucial information such as the general ledger account numbers, amounts, currencies, and other key data needed to represent financial postings in a detailed and structured manner. As one of the primary tables, BSEG plays a vital role in providing a transparent and comprehensive overview of the organization's financial data for reporting, analysis, and auditing purposes.
 
Sample 2 of Expected Answer:
A dimension table containing information about cities, including their names, states, postal codes, countries, and territories. It is used to map geographical data.
 
!!!IMPORTANT, DO NOT REPEAT THE TABLE'S NAME AS PART OF YOUR ANSWER, ANSWER STRAIGHT AWAY!!!
 
Below is the actual table's name, type (fact or dimension table), and column names as context:
Table Name: mag_dataasset3_20240216_1035
Table Type: fact
Column Names: booking_year|booking_quarter|booking_month|booking_date|booking_day|owner_login_city_code|owner_login_duty_code|owner_login_initials|owner_office_agent_type|cnt_pnr_generated|cnt_tix_sold
Answer (maximum 5 sentences of brief and concise description of the table):
"""

completion = client.chat.completions.create(
    model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    max_tokens=100,
    temperature=0.1,
)

print(completion.choices[0].message.content)

A fact table storing booking data, including the year, quarter, month, and date of bookings. It contains information about the owner who made the booking, such as their login city code, duty code, and initials. The table also tracks the office agent type and booking activity metrics, including the number of passenger name records (PNRs) generated and tickets sold. This data provides insights into booking patterns and agent performance. It is used for analyzing and reporting on booking activities.


# Mixtral 8X7B v0.1 Instruct (vLLM)

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url="http://172.210.166.80:8000/v1",
    api_key="token-abc123",
)

system_prompt = "You are a helpful assistant"

user_prompt = """"
Your task is to write sentences (maximum 5 sentences) to describe a table.
Make sure that the sentences you generated are brief, concise, and contain only essential information.
 
Sample 1 of Expected Answer:
A fact table from SAP that is a core component of the Financial Accounting (FI) module, responsible for storing individual line item data related to financial transactions. It holds crucial information such as the general ledger account numbers, amounts, currencies, and other key data needed to represent financial postings in a detailed and structured manner. As one of the primary tables, BSEG plays a vital role in providing a transparent and comprehensive overview of the organization's financial data for reporting, analysis, and auditing purposes.
 
Sample 2 of Expected Answer:
A dimension table containing information about cities, including their names, states, postal codes, countries, and territories. It is used to map geographical data.
 
!!!IMPORTANT, DO NOT REPEAT THE TABLE'S NAME AS PART OF YOUR ANSWER, ANSWER STRAIGHT AWAY!!!
 
Below is the actual table's name, type (fact or dimension table), and column names as context:
Table Name: mag_dataasset3_20240216_1035
Table Type: fact
Column Names: booking_year|booking_quarter|booking_month|booking_date|booking_day|owner_login_city_code|owner_login_duty_code|owner_login_initials|owner_office_agent_type|cnt_pnr_generated|cnt_tix_sold
Answer (maximum 5 sentences of brief and concise description of the table):
"""

completion = client.chat.completions.create(
    model="TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ",
    # prompt=f"<s> [INST] {system_prompt} [/INST] </s> [INST] {user_prompt} [/INST]",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    max_tokens=100,
    temperature=0.1,
)

print(completion.choices[0].message.content)

 This fact table records data for a specific date in 2024, containing booking information for various entities. It includes details such as the booking year, quarter, month, date, and day. Additionally, it tracks data for the city code, duty code, initials of the owner, office agent type, number of PNRs generated, and number of tickets sold. This table supports transactional and aggregated analysis in a data warehouse.


# Llama 2 70B Chat 4-Bit (vLLM using OpenAI, Non-Streaming)

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="http://172.210.166.80:8001/v1",
    api_key="token-abc123",
)

system_prompt = "You are a helpful assistant"

user_prompt = """
Your task is to write sentences (maximum 5 sentences) to describe a table.
Make sure that the sentences you generated are brief, concise, and contain only essential information.
 
Sample 1 of Expected Answer:
A fact table from SAP that is a core component of the Financial Accounting (FI) module, responsible for storing individual line item data related to financial transactions. It holds crucial information such as the general ledger account numbers, amounts, currencies, and other key data needed to represent financial postings in a detailed and structured manner. As one of the primary tables, BSEG plays a vital role in providing a transparent and comprehensive overview of the organization's financial data for reporting, analysis, and auditing purposes.
 
Sample 2 of Expected Answer:
A dimension table containing information about cities, including their names, states, postal codes, countries, and territories. It is used to map geographical data.
 
!!!IMPORTANT, DO NOT REPEAT THE TABLE'S NAME AS PART OF YOUR ANSWER, ANSWER STRAIGHT AWAY!!!
 
Below is the actual table's name, type (fact or dimension table), and column names as context:
Table Name: mag_dataasset3_20240216_1035
Table Type: fact
Column Names: booking_year|booking_quarter|booking_month|booking_date|booking_day|owner_login_city_code|owner_login_duty_code|owner_login_initials|owner_office_agent_type|cnt_pnr_generated|cnt_tix_sold
Answer (maximum 5 sentences of brief and concise description of the table):
"""

completion = client.completions.create(
    model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    prompt="""[INST] <<SYS>>
{system_prompt}
<</SYS>>
{user_prompt}[/INST]""".format(system_prompt=system_prompt, user_prompt=user_prompt),
    max_tokens=100,
    temperature=0.1,
    extra_body={"guided_choice": ["dimension", "fact"]},
)

print(completion.choices[0].text)

fact


# Llama 2 70B Chat 4-Bit (vLLM using OpenAI, Streaming)

In [37]:
from openai import OpenAI

client = OpenAI(
    base_url="http://52.191.118.16:8001/v1",
    api_key="token-abc123",
)

system_prompt = "You are a helpful assistant"

user_prompt = """
Your task is to write sentences (maximum 5 sentences) to describe a table.
Make sure that the sentences you generated are brief, concise, and contain only essential information.
 
Sample 1 of Expected Answer:
A fact table from SAP that is a core component of the Financial Accounting (FI) module, responsible for storing individual line item data related to financial transactions. It holds crucial information such as the general ledger account numbers, amounts, currencies, and other key data needed to represent financial postings in a detailed and structured manner. As one of the primary tables, BSEG plays a vital role in providing a transparent and comprehensive overview of the organization's financial data for reporting, analysis, and auditing purposes.
 
Sample 2 of Expected Answer:
A dimension table containing information about cities, including their names, states, postal codes, countries, and territories. It is used to map geographical data.
 
!!!IMPORTANT, DO NOT REPEAT THE TABLE'S NAME AS PART OF YOUR ANSWER, ANSWER STRAIGHT AWAY!!!
 
Below is the actual table's name, type (fact or dimension table), and column names as context:
Table Name: mag_dataasset3_20240216_1035
Table Type: fact
Column Names: booking_year|booking_quarter|booking_month|booking_date|booking_day|owner_login_city_code|owner_login_duty_code|owner_login_initials|owner_office_agent_type|cnt_pnr_generated|cnt_tix_sold
Answer (maximum 5 sentences of brief and concise description of the table):
"""

response = client.completions.create(
    model="TheBloke/Llama-2-70B-Chat-GPTQ",
    prompt="""[INST] <<SYS>>
{system_prompt}
<</SYS>>
{user_prompt}[/INST]""".format(system_prompt=system_prompt, user_prompt=user_prompt),
    max_tokens=100,
    temperature=0.1,
    stream=True,
)

for chunk in response:
    print(chunk.choices[0].text, end="")

  This table, mag_dataasset3_20240216_1035, is a fact table that stores data related to bookings, including the year, quarter, month, date, day, and other information related to the owner's login details and office agent type. It also contains information on the number of PNRs generated and tickets sold. This table is essential for financial reporting, analysis, and auditing purposes, as it provides a

# Llama 2 70B Chat 4-Bit (vLLM using Langchain)

In [40]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

inference_server_url = "http://52.191.118.16:8001/v1"

chat = ChatOpenAI(
    model="TheBloke/Llama-2-70B-Chat-GPTQ",
    openai_api_key="EMPTY",
    openai_api_base=inference_server_url,
    max_tokens=20,
    temperature=0.1,
)

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to Italian."
    ),
    HumanMessage(
        content="Translate the following sentence from English to Italian: I love programming."
    ),
]
chat.invoke(messages)

AIMessage(content='  Sure, I\'d be happy to help! The translation of "I love programming" from', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 48, 'total_tokens': 68}, 'model_name': 'TheBloke/Llama-2-70B-Chat-GPTQ', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None})